In [1]:
import pandas as pd
import sqlalchemy as sa
import yfinance as yf
import sys

connection_url = sa.engine.URL.create(
    "mssql+pyodbc",
    username="adfadmin",
    password="ert-Y751-vcD",
    host="analytics100sqlserv.database.windows.net",
    port=1433,
    database="analytics100sql",
    query={
        "driver": "ODBC Driver 18 for SQL Server"
        #"TrustServerCertificate": "yes",
        #"authentication": "ActiveDirectoryIntegrated",
    },
)

engine = sa.create_engine(connection_url)


In [2]:
msft = yf.Ticker('MSFT')
msft_hist = msft.history(period='max', interval='1d')
msft_hist['Date'] = msft_hist.index.strftime('%Y-%m-%d')
msft_hist['Ticker'] = 'MSFT'
msft_hist = msft_hist.reset_index(drop=True)

In [30]:
msft_hist.to_sql('History', if_exists='replace', index=False, con=engine, chunksize=1000)

-10

In [3]:
sys.getsizeof(msft_hist) / 1000000

1.701056